# EDA : 공공자전거 수요지수 기반 자치구 선별 

[목차]
1. 문제정의
    - EDA 목적
    - 용어 정의
2. 라이브러리 호출 및 기본 설정
    - 분석 환경 설정
    - 데이터 프레임 생성
2. 데이터 전처리
    - 데이터 병합
    - 결측치 처리
    - 이상치 처리
3. 수요 지수가 높은 자치구 선별
    - 필요한 데이터 선별
    - 수요지수 정의 및 자치구별 수요지수 컬럼 추가
    - 자치구별 수요지수 정규화
4. 결론

---
---

## 문제정의
- 대여하려는 사람은 많은데 대여 가능한 자전거가 부족한 따릉이 대여소가 존재한다.

### 목적
- 세부적으로 분석해 볼 자치구 선별

### 용어 정의
- 수요 : 자치구별 평균 대여수
- 공급 : 자치구별 평균 거치대 수
- `수요지수` = (거치대 수 - 대여 가능 자전거 수)/(거치대수)
    - `수요지수`가 높을수록 수요가 많은 자치구
    - 기본 자전거 수량 대비 얼마나 많은 자전거가 이용 중인가를 나타내는 지수

---
---

## 라이브러리 호출 및 기본 설정(데이터 프레임 생성 포함)

### 1) 분석 환경 설정

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [42]:
# 한글 폰트 설정
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] =False

# Pandas 보기 옵션 지정
pd.set_option('display.max_columns',100)

### 2) 데이터 프레임 생성

In [43]:
# 일시별 따릉이 대여가능 자전거 데이터 불러오기
place_df = pd.read_csv("rental_place_available_combined_data.csv",encoding='CP949', header=None)
place_df.head(3)

,0,1,2,3,4
0,2023-01-01,102,102. 망원역 1번출구 앞,0,15
1,2023-01-01,103,103. 망원역 2번출구 앞,0,6
2,2023-01-01,104,104. 합정역 1번출구 앞,0,0


In [44]:
# 컬럼몆 지정
# 거치대수량 = 대여가능 자전거 수량
place_df.columns=['일시', '대여소 번호','보관소(대여소)명', '시간대','거치대수량']

In [45]:
place_df.head(3)

,일시,대여소 번호,보관소(대여소)명,시간대,거치대수량
0,2023-01-01,102,102. 망원역 1번출구 앞,0,15
1,2023-01-01,103,103. 망원역 2번출구 앞,0,6
2,2023-01-01,104,104. 합정역 1번출구 앞,0,0


In [46]:
# 따릉이 대여소별 자치구, 거치대수 데이터 파일 불러오기
district_info_df = pd.read_csv("bike_info_cp949.csv",encoding='cp949')
district_info_df.head(3)

,Unnamed: 0,대여소 번호,보관소(대여소)명,자치구,거치대수
0,0,301,경복궁역 7번출구 앞,종로구,20
1,1,102,망원역 1번출구 앞,마포구,15
2,2,103,망원역 2번출구 앞,마포구,14


---
---

## 데이터 전처리

### 1) 데이터 병합 - 시간대별 대여소별 거치대수 대여가능 자전거수 + 대여소별 자치구 정보
- 자치구 정보를 추가하기 위함

In [47]:
# 자치구, 대여소 번호 merge
merge_df = pd.merge(place_df,district_info_df[['대여소 번호','자치구','거치대수']],on='대여소 번호')
merge_df.head(3)

,일시,대여소 번호,보관소(대여소)명,시간대,거치대수량,자치구,거치대수
0,2023-01-01,102,102. 망원역 1번출구 앞,0,15,마포구,15
1,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
2,2023-01-01,104,104. 합정역 1번출구 앞,0,0,마포구,13


In [48]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136158499 entries, 0 to 136158498
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   일시         object
 1   대여소 번호     int64 
 2   보관소(대여소)명  object
 3   시간대        int64 
 4   거치대수량      int64 
 5   자치구        object
 6   거치대수       int64 
dtypes: int64(4), object(3)
memory usage: 7.1+ GB


#### 1-1) 병합한 데이터에서 과도하게 집계된 데이터 삭제 

엑셀로 데이터를 살펴보았을 때, 102번 대여소만 동일한 날에 시간대 0 데이터가 과도하게 집계되어있는 현상 발견
- 다른 대여소는 동일한 날, 동일한 시간대별 데이터 6개씩 존재(10분 단위)
- pandas 데이터 프레임을 통해 확실하게 확인 후 102번 대여소 데이터 제거 결정됨
    - 일부 값을 남기더라도 정확한 데이터가 아닐 가능성이 높다고 판단했기 때문

In [49]:
# 데이터 이상값 확인(엑셀로 확인하고 pandas로 확실하게 확인)
# 하나의 대여소, 하나의 시간대에 10분씩 총 6개의 데이터가 존재해야함
# 102번 대여소에서 과도하게 데이터가 집계되는 이상 현상 확인

remove = merge_df.groupby(['일시','대여소 번호','시간대']).count()[['거치대수']]
remove[(remove['거치대수']!=6)&(remove['거치대수']>10)]

,,,거치대수
일시,대여소 번호,시간대,
2023-01-01,102,0,576
2023-01-16,102,0,616
2023-02-01,102,0,579
2023-02-16,102,0,501
2023-03-01,102,0,580
2023-03-16,102,0,619
2023-04-01,102,0,582
2023-04-16,102,0,582
2023-05-01,102,0,583


In [50]:
# 102번 삭제
merge_df = merge_df[merge_df['대여소 번호']!=102]
merge_df.sort_values(by=['대여소 번호','일시','시간대'],ascending=True).head(10)

,일시,대여소 번호,보관소(대여소)명,시간대,거치대수량,자치구,거치대수
1,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
2638,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
5275,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
7912,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
10550,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
13187,2023-01-01,103,103. 망원역 2번출구 앞,0,5,마포구,14
15824,2023-01-01,103,103. 망원역 2번출구 앞,1,5,마포구,14
18461,2023-01-01,103,103. 망원역 2번출구 앞,1,6,마포구,14
21099,2023-01-01,103,103. 망원역 2번출구 앞,1,6,마포구,14
23736,2023-01-01,103,103. 망원역 2번출구 앞,1,6,마포구,14


### 2) 결측치 처리

In [51]:
# 결측치 확인
(merge_df.isna().sum() / len(merge_df)) * 100

일시           0.0
대여소 번호       0.0
보관소(대여소)명    0.0
시간대          0.0
거치대수량        0.0
자치구          0.0
거치대수         0.0
dtype: float64

- 결측치가 없어 별도 처리 과정 진행 X

### 3) 이상치 처리

In [52]:
# 사분위수를 활용하여 이상치 수 확인
q1 = merge_df['거치대수량'].quantile(0.25)
q3 = merge_df['거치대수량'].quantile(0.75)
IQR = q3-q1

len(merge_df[(merge_df['거치대수량'] < (q1 - 1.5 * IQR)) | (merge_df['거치대수량'] > (q3 + 1.5 * IQR))])

6479026

In [53]:
# 이상치가 실제 이상치인지 판단하기 위해 q1,q3 확인
q1-1.5*IQR,q3 + 1.5 * IQR

(np.float64(-17.5), np.float64(34.5))

- q1-1.5*IQR 값이 음수이므로 낮은 쪽 이상치는 존재하지 않는다.
- q3+1.5*IQR 값 이상이어도 충분히 실제로 존재할 수 있는 값이기 때문에 max값과 대여소별 평균 대여가능 자전거 수량을 확인해본다. 

In [54]:
# 거치대 수량 최댓값 > 421
max(merge_df['거치대수량'])

421

In [55]:
# MAX값이 나온 대여소 : '2037. 동작역 5번출구 동작주차공원'
merge_df[merge_df['거치대수량']==421]['보관소(대여소)명'].unique()

array(['2037. 동작역 5번출구 동작주차공원'], dtype=object)

- '2037. 동작역 5번출구 동작주차공원' 대여소 확인 결과 한강 공원 근처에 위치하여 실제로 공급량과 수요량 모두 많고, 몇백개씩 쌓이는 현상이 발생하는 것으로 확인되었기 때문에 이상치라고 판단하지 않음

In [56]:
# 평균 대여소별 대여가능 자전거 수량의 최댓값이 50 > 충분히 존재할 수 있는 값
merge_df[['보관소(대여소)명','거치대수량']].groupby('보관소(대여소)명').mean(numeric_only=True).describe()

,거치대수량
count,2719.000000
mean,10.544725
std,7.206146
min,0.000000
25%,5.373312
50%,8.806500
75%,14.114862
max,50.911153


- 이상치를 삭제하게 되면 과한 공급 케이스를 삭제하여 공급의 불균형 현상을 왜곡할 수 있기 때문에 따로 삭제하지 않고 분석 진행
    - 한강 근처 대여소일 경우 공급량과 수요량 모두 많기 때문에 높은 수치가 나와도 이상치라고 판단하지 않는다.

In [57]:
merge_df.head(3)

,일시,대여소 번호,보관소(대여소)명,시간대,거치대수량,자치구,거치대수
1,2023-01-01,103,103. 망원역 2번출구 앞,0,6,마포구,14
2,2023-01-01,104,104. 합정역 1번출구 앞,0,0,마포구,13
3,2023-01-01,105,105. 합정역 5번출구 앞,0,1,마포구,5


---
---

## 수요 지수가 높은 자치구 선별

### 1) 필요한 데이터 선별

In [58]:
# 자치구별 평균 대여가능 자전거 / 거치대수 데이터 프레임 생성
result = merge_df.groupby(['자치구']).mean(numeric_only=True)[['거치대수량','거치대수']]
result.head(3)

,거치대수량,거치대수
자치구,,
강남구,6.410732,10.566483
강동구,12.308702,12.597430
강북구,7.308240,10.709306


### 2) 수요지수 정의 및 자치구별 수요지수 컬럼 추가

In [59]:
# 수요지수 컬럼 생성
result['대여수'] = result['거치대수'] - result['거치대수량']
result['수요지수'] = (result['거치대수'] - result['거치대수량'])/result['거치대수']

In [60]:
# 수요지수 TOP3 자치구 선정
result = result.sort_values(by='수요지수', ascending=False)
result.head(3)

,거치대수량,거치대수,대여수,수요지수
자치구,,,,
서대문구,7.303207,12.050333,4.747126,0.393942
강남구,6.410732,10.566483,4.155751,0.393296
중구,7.111569,11.418966,4.307397,0.377214


---
---

### 3) 자치구별 수요지수 정규화

In [61]:
# 자치구별 수요지수
demand_df = result.sort_values(by='수요지수',ascending=False)
demand_df

,거치대수량,거치대수,대여수,수요지수
자치구,,,,
서대문구,7.303207,12.050333,4.747126,0.393942
강남구,6.410732,10.566483,4.155751,0.393296
중구,7.111569,11.418966,4.307397,0.377214
관악구,7.541627,12.065457,4.523830,0.374941
서초구,7.627786,11.980420,4.352634,0.363312
용산구,8.276958,12.314879,4.037921,0.327890
종로구,7.834931,11.600000,3.765069,0.324575
강북구,7.308240,10.709306,3.401065,0.317580
송파구,8.913311,12.567692,3.654380,0.290776


- `수요 지수` 정규화 진행

In [62]:
# 정규화 진행
scaler = StandardScaler()
demand_df["수요지수_정규화"] = scaler.fit_transform(demand_df[["수요지수"]])

In [63]:
# 정규화된 수요지수 TOP3 자치구
demand_df = demand_df.sort_values(by='수요지수_정규화',ascending=False)
demand_df.head(3)

,거치대수량,거치대수,대여수,수요지수,수요지수_정규화
자치구,,,,,
서대문구,7.303207,12.050333,4.747126,0.393942,1.171994
강남구,6.410732,10.566483,4.155751,0.393296,1.169060
중구,7.111569,11.418966,4.307397,0.377214,1.096018


#### 자치구별 수요지수 시각화를 위한 파일 Export

In [64]:
demand_export_df = demand_df.reset_index()

In [65]:
demand_export_df[['자치구','수요지수', '수요지수_정규화']]

,자치구,수요지수,수요지수_정규화
0,서대문구,0.393942,1.171994
1,강남구,0.393296,1.169060
2,중구,0.377214,1.096018
3,관악구,0.374941,1.085691
4,서초구,0.363312,1.032874
5,용산구,0.327890,0.871982
6,종로구,0.324575,0.856926
7,강북구,0.317580,0.825156
8,송파구,0.290776,0.703408
9,도봉구,0.275406,0.633599


In [66]:
demand_export_df.to_csv("자치구별_수요지수_시각화_데이터.csv", encoding='utf-8')

---
## 결론

가장 따릉이 수요가 많은 자치구
    
- '`서대문구`','`강남구`','`중구`'

따라서, 서대문구/강남구/중구 대상 세부적인 추가 분석을 실시하기로 한다.
- 각 자치구에서 수요지수가 높은 대여소 10개 선별
- 선별한 대여소의 시간대별 대여가능 자전거 수량 변화 추이를 확인하여 수요 불균형 발생하는 대여소 최종 선별
- 최종 선별된 대여소 근처 인프라, 상권, 교통 상황, 인구 밀도 등을 고려하여 해결 인사이트 도출

---
---